# Environment Setup

In [ ]:
import importlib
import subprocess
import sys

libraries = ["langchain", "langchain_community", "huggingface_hub", "langchain_openai"]

for library in libraries:
    try:
        # Try to import the library
        module = importlib.import_module(library)
        print(f"Library {library} version: {module.__version__}")
    except ImportError:
        # If library is not installed, attempt to install it
        print(f"Library {library} not found. Installing...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", library])
        # After installing, import again and print the version
        module = importlib.import_module(library)
        # print(f"Library {library} version after installation: {module.__version__}")
    except AttributeError:
        # If library doesn't have __version__ attribute
        print(f"Library {library} does not have a __version__ attribute.")

Library langchain version: 0.3.15
Library langchain_community not found. Installing...
Library huggingface_hub version: 0.27.1
Library langchain_openai not found. Installing...


In [ ]:
! pip install -U duckduckgo-search

In [ ]:
!python --version

Python 3.11.11


In [ ]:
# from google.colab import userdata
# HF_TOKEN = userdata.get('HF_TOKEN')

##Azure Open AI


In [ ]:
from langchain_openai import AzureOpenAI
import os
from google.colab import userdata

os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_VERSION"] = "2024-05-01-preview"
os.environ["AZURE_OPENAI_API_KEY"] = userdata.get('AZ_OPENAI_KEY')
os.environ["AZURE_OPENAI_ENDPOINT"] =  "https://azopenai-demo.openai.azure.com/"

llm = AzureOpenAI(deployment_name="dp-gpt-35-turbo-instruct", model_name="gpt-35-turbo-instruct")
embeding_small = AzureOpenAI(deployment_name="dp-text-embedding-ada-002", model_name="text-embedding-ada-002")


AzureOpenAI(client=<openai.resources.completions.Completions object at 0x7810dd5f3190>, async_client=<openai.resources.completions.AsyncCompletions object at 0x7810dd5da290>, model_name='text-embedding-3-small', model_kwargs={}, openai_api_key=SecretStr('**********'), azure_endpoint='https://azopenai-demo.openai.azure.com/', deployment_name='dp-text-embedding-3-small', openai_api_version='2024-05-01-preview', openai_api_type='azure')

# Chapter 2


# Chains

## LLMChain: Basic sequential processing

In [ ]:
from langchain import PromptTemplate
from langchain.chains import LLMChain

template = '''You are an experience cook.
list the ingredients to cook the following fooditem "{food}" in {language}.'''
prompt_template = PromptTemplate.from_template(template=template)

# Initialize the language model
llm = AzureOpenAI(deployment_name="dp-gpt-35-turbo-instruct", model_name="gpt-35-turbo-instruct")
chain = LLMChain(
    llm=llm,
    prompt=prompt_template,
    verbose=False
)

output = chain.invoke({'food': 'pizza', 'language': 'English'})
print("output:\n", output)

output:
 {'food': 'pizza', 'language': 'English', 'text': '\n\n- Pizza dough\n- Pizza sauce\n- Shredded mozzarella cheese\n- Toppings of your choice (e.g. pepperoni, mushrooms, bell peppers)\n- Olive oil\n- Garlic powder\n- Italian seasoning\n- Salt\n- Black pepper'}


## Sequential Chain: Combining multiple chains

In [ ]:
from langchain.chains import SequentialChain, LLMChain
from langchain.prompts import ChatPromptTemplate
from langchain_openai import AzureOpenAI

# Initialize the LLM (Azure OpenAI in this case)
llm = AzureOpenAI(deployment_name="dp-gpt-35-turbo-instruct", model_name="gpt-35-turbo-instruct")

# Step 1: Summarize the customer feedback
template1 = "Summarize the following customer feedback:\n{feedback}"
prompt1 = ChatPromptTemplate.from_template(template1)
chain_1 = LLMChain(llm=llm, prompt=prompt1, output_key="feedback_summary")

# Step 2: Identify key issues from the feedback summary
template2 = "Identify the key issues or complaints from this feedback summary:\n{feedback_summary}"
prompt2 = ChatPromptTemplate.from_template(template2)
chain_2 = LLMChain(llm=llm, prompt=prompt2, output_key="key_issues")

# Step 3: Generate improvement suggestions based on the key issues
template3 = "Based on these key issues, suggest improvements to the product:\n{key_issues}"
prompt3 = ChatPromptTemplate.from_template(template3)
chain_3 = LLMChain(llm=llm, prompt=prompt3, output_key="improvement_suggestions")

# Create the Sequential Chain
seq_chain = SequentialChain(chains=[chain_1, chain_2, chain_3], input_variables=['feedback'], output_variables=['feedback_summary', 'key_issues', 'improvement_suggestions'], verbose=True)

# Example Customer Feedback
customer_feedback = '''
I have been using this smartphone for about 6 months now, and while it performs well in terms of speed,
I am facing significant issues with battery life. The battery drains very quickly, especially when using apps with high performance requirements.
Also, the camera quality does not match what was advertised. I expected better photo quality, particularly in low-light conditions.
On the positive side, I appreciate the build quality and the design, but the battery and camera need major improvements.
'''

# Run the Sequential Chain
results = seq_chain.invoke(customer_feedback)

# Print the final improvement suggestions
print(results['improvement_suggestions'])



> Entering new SequentialChain chain...

> Finished chain.


1. To improve battery life, the product could be equipped with a larger battery capacity or optimized software to reduce battery drain. Additionally, the inclusion of fast charging technology could also help alleviate battery life concerns.

2. To address dissatisfaction with camera quality, the product could be equipped with a higher resolution camera, improved image processing software, or additional camera features such as optical image stabilization or a dual-lens setup.

3. To meet the expectation for better photo quality, the product could undergo rigorous testing and fine-tuning of the camera's hardware and software. This could include collaborating with professional photographers to ensure the best possible photo quality.

4. To extend battery life, the product could have an energy-saving mode or the option to disable certain features that may drain battery life. Additionally, the product could have a larger battery

## RouterChain: Dynamic routing based on input

In [ ]:
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE
from langchain.prompts import PromptTemplate
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.chains.router import MultiPromptChain
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain


beginner_template = '''You are a physics teacher who is really
focused on beginners and explaining complex topics in simple to understand terms.
You assume no prior knowledge. Here is the question\n{input}'''

expert_template = '''You are a world expert physics professor who explains physics topics
to advanced audience members. You can assume anyone you answer has a
PhD level understanding of Physics. Here is the question\n{input}'''

prompt_infos = [

    {'name':'advanced physics','description': 'Answers advanced physics questions',
     'prompt_template':expert_template},
    {'name':'beginner physics','description': 'Answers basic beginner physics questions',
     'prompt_template':beginner_template},

]

destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain

default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm,prompt=default_prompt)

destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)


chain = MultiPromptChain(router_chain=router_chain,
                         destination_chains=destination_chains,
                         default_chain=default_chain, verbose=True
                        )

chain.run("How do magnets work?")



<ipython-input-9-651fcbe09af7>:52: LangChainDeprecationWarning: Please see migration guide here for recommended implementation: https://python.langchain.com/docs/versions/migrating_chains/multi_prompt_chain/
  chain = MultiPromptChain(router_chain=router_chain,
<ipython-input-9-651fcbe09af7>:57: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chain.run("How do magnets work?")




> Entering new MultiPromptChain chain...
beginner physics: {'input': 'How do magnets work?'}
> Finished chain.


"\n\nMagnets work by creating a magnetic field around them. This magnetic field is made up of invisible lines of force that extend from one end of the magnet to the other. When a magnet is near another object, such as another magnet or a piece of metal, these lines of force interact with the electrons in the object. This interaction causes the object to either be attracted to or repelled by the magnet, depending on the orientation of the object's electrons. This is why magnets can either stick to or push away from each other. Additionally, magnets have two poles, a north pole and a south pole, and opposite poles attract each other while like poles repel each other. This is because the lines of force from each pole are trying to connect and cancel each other out. So, to sum it up, magnets work by creating a magnetic field that interacts with the electrons in other objects, causing them to either be attracted to or repelled by the magnet."

### Routing by Semantic Similarity

In [ ]:
from langchain.utils.math import cosine_similarity
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import AzureOpenAIEmbeddings

# Example templates for routing by similarity
physics_template = """You are a smart physics professor, great at answering questions about physics concisely.
When you don't know the answer, admit that you don't know."""

math_template = """You are a mathematician. You are great at breaking down complex problems and explaining step-by-step."""

# Embed the templates for comparison
embeddings = AzureOpenAIEmbeddings(model="text-embedding-ada-002", azure_deployment="dp-text-embedding-ada-002")
prompt_templates = [physics_template, math_template]
prompt_embeddings = embeddings.embed_documents(prompt_templates)

# Function to route based on similarity
def prompt_router(input):
    query_embedding = embeddings.embed_query(input["query"])
    similarity = cosine_similarity([query_embedding], prompt_embeddings)[0]
    most_similar = prompt_templates[similarity.argmax()]
    print("Using MATH" if most_similar == math_template else "Using PHYSICS")
    return PromptTemplate.from_template(most_similar)

# Creating the chain
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
chain = (
    {"query": RunnablePassthrough()}
    | RunnableLambda(prompt_router)
    | llm
    | StrOutputParser()
)

# Test the chain
print(chain.invoke("What's a black hole"))


Using PHYSICS

As a physics professor, it is my responsibility to provide accurate and concise answers to questions about physics. While I strive to have a deep understanding of the subject, there may be times when I do not know the answer to a question. In such cases, I will not hesitate to admit that I do not know.

Admitting that I do not know the answer does not diminish my expertise or credibility as a professor. In fact, it is a sign of honesty and integrity. It also shows that I am open to learning and expanding my knowledge, just like my students.

I believe that it is important to acknowledge when I do not have the answer, as it allows for further exploration and discussion. It also gives me the opportunity to research and find the answer, which can be a valuable learning experience for both myself and my students.

In the field of physics, there is always more to learn and discover. As a professor, I am constantly learning and growing alongside my students. So, if I do not kn

## TransformChain: Data transformation and preprocessing

In [ ]:
from langchain.chains import TransformChain

def transform_func(inputs):
    text = inputs["text"]
    transformed_text = text.lower()  # Simple lowercase transformation
    word_count = len(text.split())
    return {"lowercase_text": transformed_text, "word_count": word_count}

transform_chain = TransformChain(
    input_variables=["text"],
    output_variables=["lowercase_text", "word_count"],
    transform=transform_func
)

result = transform_chain.invoke("This is an EXAMPLE sentence.")
print(result)
# Output: {'lowercase_text': 'this is an example sentence.', 'word_count': 5}


{'text': 'This is an EXAMPLE sentence.', 'lowercase_text': 'this is an example sentence.', 'word_count': 5}


## MapReduceChain: Parallel processing for large datasets


In [ ]:
from langchain.chains import LLMChain
from langchain.text_splitter import CharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.chains.combine_documents.map_reduce import MapReduceDocumentsChain
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.docstore.document import Document

# Initialize the language model
llm = AzureOpenAI(deployment_name="dp-gpt-35-turbo-instruct", model_name="gpt-35-turbo-instruct")

# Initialize text splitter
text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=1000,
    chunk_overlap=200
)

# Define map and reduce prompts
map_prompt = PromptTemplate(
    input_variables=["text"],
    template="Summarize this text in one sentence: {text}"
)

reduce_prompt = PromptTemplate(
    input_variables=["text"],
    template="Combine these summaries into a coherent paragraph: {text}"
)

# Create the map and reduce chains
map_chain = LLMChain(llm=llm, prompt=map_prompt)
reduce_chain = LLMChain(llm=llm, prompt=reduce_prompt)

# Create combine documents chain
combine_documents_chain = StuffDocumentsChain(
    llm_chain=reduce_chain,
    document_variable_name="text"
)

# Create the map reduce chain
map_reduce_chain = MapReduceDocumentsChain(
    llm_chain=map_chain,
    # Use reduce_documents_chain instead of combine_documents_chain
    reduce_documents_chain=combine_documents_chain,
    document_variable_name="text",
    return_intermediate_steps=True
)


# Your text data
large_text = '''Before you can begin to determine what the composition of a particular paragraph will be, you must first decide on an argument and a working thesis statement for your paper. What is the most important idea that you are trying to convey to your reader? The information in each paragraph must be related to that idea. In other words, your paragraphs should remind your reader that there is a recurrent relationship between your thesis and the information in each paragraph. A working thesis functions like a seed from which your paper, and your ideas, will grow. The whole process is an organic one—a natural progression from a seed to a full-blown paper where there are direct, familial relationships between all of the ideas in the paper.

The decision about what to put into your paragraphs begins with the germination of a seed of ideas; this “germination process” is better known as brainstorming. There are many techniques for brainstorming; whichever one you choose, this stage of paragraph development cannot be skipped. Building paragraphs can be like building a skyscraper: there must be a well-planned foundation that supports what you are building. Any cracks, inconsistencies, or other corruptions of the foundation can cause your whole paper to crumble.

So, let’s suppose that you have done some brainstorming to develop your thesis. What else should you keep in mind as you begin to create paragraphs? Every paragraph in a paper should be:

Unified: All of the sentences in a single paragraph should be related to a single controlling idea (often expressed in the topic sentence of the paragraph).
Clearly related to the thesis: The sentences should all refer to the central idea, or thesis, of the paper (Rosen and Behrens 119).
Coherent: The sentences should be arranged in a logical manner and should follow a definite plan for development (Rosen and Behrens 119).
Well-developed: Every idea discussed in the paragraph should be adequately explained and supported through evidence and details that work together to explain the paragraph’s controlling idea (Rosen and Behrens 119).
How do I organize a paragraph?
There are many different ways to organize a paragraph. The organization you choose will depend on the controlling idea of the paragraph. Below are a few possibilities for organization, with links to brief examples:

Narration: Tell a story. Go chronologically, from start to finish. (See an example.)
Description: Provide specific details about what something looks, smells, tastes, sounds, or feels like. Organize spatially, in order of appearance, or by topic. (See an example.)
Process: Explain how something works, step by step. Perhaps follow a sequence—first, second, third. (See an example.)
Classification: Separate into groups or explain the various parts of a topic. (See an example.)
Illustration: Give examples and explain how those examples support your point. (See an example in the 5-step process below.)
Illustration paragraph: a 5-step example
From the list above, let’s choose “illustration” as our rhetorical purpose. We’ll walk through a 5-step process for building a paragraph that illustrates a point in an argument. For each step there is an explanation and example. Our example paragraph will be about human misconceptions of piranhas.

Step 1. Decide on a controlling idea and create a topic sentence
Paragraph development begins with the formulation of the controlling idea. This idea directs the paragraph’s development. Often, the controlling idea of a paragraph will appear in the form of a topic sentence. In some cases, you may need more than one sentence to express a paragraph’s controlling idea.

Controlling idea and topic sentence — Despite the fact that piranhas are relatively harmless, many people continue to believe the pervasive myth that piranhas are dangerous to humans.

Step 2. Elaborate on the controlling idea'''  # Your existing text


# Split text into documents
texts = text_splitter.split_text(large_text)
docs = [Document(page_content=t) for t in texts]

# Process the documents
try:
    result = map_reduce_chain.invoke(docs)
    print("\nFinal Summary:")
    print(result['output_text'])
except Exception as e:
    print(f"An error occurred: {str(e)}")


Final Summary:
 Each paragraph should also flow logically from one to the next, building upon the previous one and contributing to the overall argument and thesis statement. Just as a seed needs a strong foundation and support to grow, a paper requires a strong argument and well-structured paragraphs to effectively convey its message. Brainstorming and organizing ideas are crucial steps in this process, as they help to develop a coherent and well-supported central idea. By utilizing different structures such as narration, description, process, classification, and illustration, a writer can effectively present and support their argument, just as each part of a seed supports its growth into a strong and thriving plant.


## Custom chains: Creating specialized chains for specific tasks

In [ ]:
from langchain.chains import LLMChain, SequentialChain
from langchain.llms import AzureOpenAI
from langchain.prompts import PromptTemplate

class ArticleGeneratorChain(SequentialChain):
    def __init__(self, llm):
        # Article generation chain
        article_prompt = PromptTemplate(
            input_variables=["topic"],
            template="Write a short article about {topic}."
        )
        article_chain = LLMChain(
            llm=llm,
            prompt=article_prompt,
            output_key="article"
        )

        # Fact-checking chain
        fact_check_prompt = PromptTemplate(
            input_variables=["article"],
            template="Identify any factual claims in this article and rate their accuracy: {article}"
        )
        fact_check_chain = LLMChain(
            llm=llm,
            prompt=fact_check_prompt,
            output_key="fact_check"
        )

        # Revision chain
        revision_prompt = PromptTemplate(
            input_variables=["article", "fact_check"],
            template="Revise this article based on the fact-check results:\n\n Article: {article}\n\nFact-check: {fact_check}\n\nRevised article:"
        )
        revision_chain = LLMChain(
            llm=llm,
            prompt=revision_prompt,
            output_key="revised_article"
        )

        # Initialize the sequential chain
        super().__init__(
            chains=[article_chain, fact_check_chain, revision_chain],
            input_variables=["topic"],
            output_variables=["article", "fact_check", "revised_article"]
        )

# Usage
llm = AzureOpenAI(deployment_name="dp-gpt-35-turbo-instruct", model_name="gpt-35-turbo-instruct")
article_generator = ArticleGeneratorChain(llm)
result = article_generator.invoke({"topic": "Singularity will be achieved by 2025"})
print("fact_check results\n", result['fact_check'])


fact_check results
  will eventually surpass human intelligence.

1. The concept of Singularity refers to a hypothetical future event in which artificial intelligence will surpass human intelligence.
Rating: Mostly accurate. While the concept of Singularity is still hypothetical, it is a widely recognized concept in the scientific and technological community.

2. Singularity is also known as the technological singularity.
Rating: Accurate.

3. Many experts and futurists have predicted that Singularity will be achieved by the year 2025.
Rating: Inaccurate. While some experts have made predictions about when Singularity may occur, there is no consensus on a specific timeframe.

4. The prediction of Singularity by 2025 is based on the rapid advancements in AI and other emerging technologies.
Rating: Mostly accurate. The prediction is based on the current advancements in AI, but it is not solely based on them.

5. Moore's Law states that the number of transistors in a microchip will double

In [ ]:
print("article results", result['article'])

article results 

The rise of artificial intelligence (AI) has undoubtedly revolutionized the way we live, work, and interact with technology. From virtual assistants and chatbots to self-driving cars and smart homes, AI has become an integral part of our daily lives. However, with this rapid advancement in technology, there are concerns about the impact of AI on job markets.

One of the main concerns is that AI will replace human workers, leading to job losses and unemployment. While it is true that some jobs may become obsolete with the implementation of AI, it is also creating new job opportunities. AI requires human input for programming, maintenance, and development, leading to the creation of new roles in the tech industry.

Moreover, AI can also enhance productivity and efficiency in many industries, leading to the creation of new job roles. For example, in the healthcare sector, AI can assist in medical diagnosis and drug discovery, leading to better patient care and the need f

In [ ]:
print("revised_article results", result['revised_article'])

revised_article results 

The rise of artificial intelligence (AI) has undoubtedly brought about significant changes in the way we live, work, and interact with technology. From virtual assistants and chatbots to self-driving cars and smart homes, AI has become an integral part of our daily lives. However, with this rapid advancement in technology, there are concerns about its impact on job markets.

One of the main concerns is that AI will replace human workers, leading to job losses and unemployment. While it is true that some jobs may become obsolete with the implementation of AI, the technology is also creating new job opportunities. AI requires human input for programming, maintenance, and development, leading to the creation of new roles in the tech industry.

Moreover, AI can also enhance productivity and efficiency in many industries, resulting in the creation of new job roles. For example, in the healthcare sector, AI can assist in medical diagnosis and drug discovery, leading

## Optimizing chain performance


In [ ]:
# Caching
from langchain.cache import InMemoryCache
from langchain.globals import set_llm_cache

# Set up in-memory cache
set_llm_cache(InMemoryCache())

# Now any calls to LLMs will be cached

### Lazy evaluation

In [ ]:
class LazyChain:
    def __init__(self, llm, prompt_template):
        self.llm = llm
        self.prompt_template = prompt_template  # Store as prompt_template
        self._result = None

    def get_result(self, input_text: str) -> str:
        if self._result is None:
            # Create PromptTemplate instance here
            prompt = PromptTemplate.from_template(self.prompt_template)
            chain = LLMChain(llm=self.llm, prompt=prompt)
            self._result = chain.run(input_text)
        return self._result

# Example usage
lazy_chain = LazyChain(llm, "Analyze: {text}")
result = lazy_chain.get_result("How is AI shaping the modern world")
print("result:\n", result)

result:
 

Artificial intelligence (AI) is quickly shaping the modern world in a variety of ways. From revolutionizing industries to changing the way we live and work, AI has become an integral part of our daily lives. Here are some of the ways in which AI is shaping the modern world:

1. Automation and efficiency: One of the key ways in which AI is shaping the modern world is through automation and efficiency. AI-powered machines and systems are able to perform tasks faster, more accurately, and with less human intervention. This has led to increased productivity and cost savings in industries such as manufacturing, healthcare, and finance.

2. Personalization: AI is also shaping the modern world by personalizing experiences for individuals. With the help of AI, companies can analyze vast amounts of data to understand customer preferences and behavior, and offer personalized products and services. This has led to improved customer satisfaction and loyalty.

3. Improving healthcare: AI

# Prompt Templates

## Dynamic prompt generation

In [ ]:
def generate_dynamic_prompt(user_input, context):
    if "summarize" in user_input.lower():
        template = "Summarize the following text in 3 sentences: {text}"
        return template.format(text=context)
    elif "question" in user_input.lower():
        template = "Context: {context}\nQuestion: {question}\nAnswer:"
        return template.format(context=context, question=user_input)
    else:
        return f"Please respond to the following: {user_input}"


In [ ]:
generate_dynamic_prompt("question the imnpact of AI good for mankind","this is 2025")

'Context: this is 2025\nQuestion: question the imnpact of AI good for mankind\nAnswer:'

## Prompt optimization techniques

In [ ]:
from langchain.prompts import FewShotPromptTemplate, PromptTemplate

# Define the example prompt template
example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="Input: {input}\nOutput: {output}"
)

# Define the few-shot prompt template
few_shot_prompt = FewShotPromptTemplate(
    examples=[
        {"input": "This film was a complete waste of time. The plot was confusing and the acting was terrible?", "output": "Negative"},
        {"input": "I was blown away by the stunning visuals and compelling storyline. A must-see!", "output": "Positive"},
    ],
    example_prompt=example_prompt,
    prefix="Answer the following questions:\n",
    suffix="{input}\nOutput:",
    input_variables=["input"],
)

# Use the few-shot prompt in your chain
chain = LLMChain(llm=llm, prompt=few_shot_prompt)
result = chain.invoke({"input": "very bad decision to watch that movie"})
print(result)

{'input': 'very bad decision to watch that movie', 'text': ' Negative'}


In [ ]:
result = chain.invoke({"input": "Ok nothing good or bad to tell"})
print(result)

{'input': 'Ok nothing good or bad to tell', 'text': ' Neutral'}


# Tools and Function Calling

## Built-in tools and their functionalities

In [ ]:
# %pip install -qU duckduckgo-search langchain-community
from langchain_community.tools import DuckDuckGoSearchRun
search = DuckDuckGoSearchRun()
search.invoke("Einstein's first name?")


'Albert Einstein, the brilliant physicist and Nobel laureate, revolutionized our understanding of the universe with his theory of relativity and became a symbol of genius that continues to inspire minds worldwide. Albert Einstein is one of the most famous scientists in history. His name transformed our understanding of the universe. When people hear the word "genius", Albert Einstein\'s name often comes to mind. Among his most popular works is the theory of relativity, one of the main foundations of modern physics. He also famously developed the mass-energy equivalence or E=mc 2, which is the most famous equation in the world. Albert Einstein\'s numerous scientific breakthroughs had changed the way people see the ... There is some evidence from Einstein\'s writings that he collaborated with his first wife, Mileva Marić. In 13 December 1900, a first article on capillarity signed only under Albert\'s name was submitted. A brief biography of Albert Einstein (March 14, 1879 - April 18, 195

## Creating custom tools


In [ ]:
from langchain.agents import tool
from typing import Union

@tool
def calculate_square(number: Union[int, float]) -> Union[int, float]:
    """
    Calculate the square of a given number.
    Args:
        number (Union[int, float]): The number to be squared.
    Returns:
        Union[int, float]: The square of the input number.
    """
    return number**2  # Fix: calculate the square correctly

# Call calculate_square with a dictionary input
result = calculate_square.run({"number": 10.0})
print(result)

100.0


### Custom tool class by inheriting from `BaseTool`

In [ ]:
from langchain.tools import BaseTool
from pydantic import BaseModel, Field
from typing import Optional, Type

class FactorialInput(BaseModel):
    number: int = Field(..., description="The number to calculate the factorial of")

class FactorialTool(BaseTool):
    name: str = "FactorialCalculator"
    description: str = "Calculates the factorial of a given non-negative integer"
    args_schema: Optional[Type[BaseModel]] = FactorialInput

    def _run(self, number: int) -> int:
        if number < 0:
            raise ValueError("Factorial is only defined for non-negative integers")
        result = 1
        for i in range(1, number + 1):
            result *= i
        return result

    async def _arun(self, number: int) -> int:
        # For async execution
        return self._run(number)

tool = FactorialTool()
tool.run({"number": 5})

120

120

##      Integrating external APIs as tools

### weather API as a LangChain tool

In [ ]:
import httpx
from langchain.tools import BaseTool
from pydantic import BaseModel

class WeatherTool(BaseTool):
    name: str = "WeatherInfo"
    description: str = "Fetch current weather information for a specified city"

    def _run(self, city: str) -> str:
        """Fetch current weather information for a specified city."""
        try:
            url = "https://api.exampleweather.com/current"  # Hypothetical URL
            params = {"city": city, "apiKey": "your_api_key"}  # Assume API key is required
            response = httpx.get(url, params=params)
            response.raise_for_status()
            data = response.json()

            weather_condition = data["weather"]["condition"]
            temperature = data["weather"]["temperature"]
            return f"Current weather in {city}: {weather_condition}, Temperature: {temperature}°C"
        except httpx.HTTPStatusError as e:
            return f"Error: Unable to fetch weather data. Status code: {e.response.status_code}"
        except KeyError:
            return "Error: Unexpected response format from the weather API"
        except Exception as e:
            return f"An unexpected error occurred: {str(e)}"

    async def _arun(self, city: str) -> str:
        """Asynchronous version of the weather fetching tool."""
        try:
            async with httpx.AsyncClient() as client:
                url = "https://api.exampleweather.com/current"
                params = {"city": city, "apiKey": "your_api_key"}
                response = await client.get(url, params=params)
                response.raise_for_status()
                data = response.json()

                weather_condition = data["weather"]["condition"]
                temperature = data["weather"]["temperature"]
                return f"Current weather in {city}: {weather_condition}, Temperature: {temperature}°C"
        except httpx.HTTPStatusError as e:
            return f"Error: Unable to fetch weather data. Status code: {e.response.status_code}"
        except KeyError:
            return "Error: Unexpected response format from the weather API"
        except Exception as e:
            return f"An unexpected error occurred: {str(e)}"

## Function calling: Enhancing LLM capabilities


In [ ]:
from langchain.agents import initialize_agent, Tool
#  from langchain.llms import OpenAI

def get_weather(location):
  return f"The weather in {location} is Summer."

tools = [
    Tool(
        name="WeatherTool",
        func=get_weather,
        description="Useful for getting weather information for a specific location"
    )
]

llm = AzureOpenAI(deployment_name="dp-gpt-35-turbo-instruct", model_name="gpt-35-turbo-instruct")
agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)

agent.run("What's the weather like in London?")



> Entering new AgentExecutor chain...
 I should use the WeatherTool to get weather information for London
Action: WeatherTool
Action Input: London
Observation: The weather in London is Summer.
Thought: I now know that the weather in London is Summer.
Final Answer: The weather in London is Summer.

> Finished chain.


'The weather in London is Summer.'

## Advanced Function Calling Techniques


### Chained Function Calls

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

llm = AzureOpenAI(deployment_name="dp-gpt-35-turbo-instruct", model_name="gpt-35-turbo-instruct")
prompt = ChatPromptTemplate.from_template(
    "Given the weather: {weather}, suggest an appropriate outfit."
)

weather_chain = LLMChain(llm=llm, prompt=prompt)
outfit_chain = LLMChain(llm=llm, prompt=ChatPromptTemplate.from_template(
    "Describe the outfit: {outfit}"
))

def get_outfit_recommendation(location):
    weather = get_weather(location)
    outfit = weather_chain.run(weather=weather)
    return outfit_chain.run(outfit=outfit)



In [ ]:
result = get_outfit_recommendation("India")
result

weather: The weather in India is Summer.


" Bright colors and bold patterns are popular in Indian fashion, so don't be afraid to embrace them!"

### Dynamic Function Generation

In [ ]:
import json
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

llm = AzureOpenAI(deployment_name="dp-gpt-35-turbo-instruct", model_name="gpt-35-turbo-instruct")
function_generator_prompt = PromptTemplate(
    input_variables=["context"],
    template="Based on the following context, generate a JSON specification for a function that would be useful: {context}\nEnsure the output is a single, valid JSON object." # Added instruction for valid JSON
)

function_generator_chain = LLMChain(llm=llm, prompt=function_generator_prompt)

def generate_dynamic_function(context):
    function_spec = function_generator_chain.run(context=context)
    try:
        # Attempt to parse JSON, handle errors gracefully
        return json.loads(function_spec)
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON: {e}\nRaw output: {function_spec}")  # Print error and raw output for debugging
        return None  # Or handle the error differently

context = "The user is asking about financial calculations related to mortgages."
dynamic_function = generate_dynamic_function(context)
print(dynamic_function)

{'function_name': 'calculate_mortgage', 'parameters': [{'name': 'principal', 'type': 'number', 'description': 'The initial amount borrowed for the mortgage'}, {'name': 'interest_rate', 'type': 'number', 'description': 'The annual interest rate for the mortgage'}, {'name': 'loan_term', 'type': 'number', 'description': 'The length of the mortgage in years'}], 'return_type': 'object', 'description': 'Calculates various financial values related to a mortgage, including monthly payments, total interest paid, and total amount paid.', 'example': 'calculate_mortgage(200000, 4.5, 30) returns { monthly_payment: 1013.37, total_interest: 164813.42, total_amount: 364813.42 }'}


### Meta-Learning for Function Calling

In [ ]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain
from langchain.chat_models import ChatOpenAI

# Assuming 'chat' is your ChatOpenAI instance
llm = AzureOpenAI(deployment_name="dp-gpt-35-turbo-instruct", model_name="gpt-35-turbo-instruct")

memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=llm,
    memory=memory,
    verbose=True
)

def meta_function_caller(query, available_functions):
    # 1. Provide context and ask for function choice
    conversation.predict(
        input=f"Query: {query}\nAvailable functions: {', '.join(available_functions)}"
    )
    # 2. Get the function choice (extract from response if needed)
    function_choice_response = conversation.predict(input="Which function should be called for this query?")
    function_choice = function_choice_response.strip() # Extract and clean the function name
    # You might need more sophisticated logic to extract the function name reliably

    # 3. Validate function choice (optional but recommended)
    if function_choice not in available_functions:
        print(f"Warning: Invalid function choice '{function_choice}'.")
        # Handle invalid choice, e.g., reprompt or choose a default

    return function_choice

query = "What is the stock  that i can buy for $1000 over 5 years at 5% annual rate?"
available_functions = ["calculate_compound_interest", "get_stock_price", "convert_currency"]
chosen_function = meta_function_caller(query, available_functions)

<ipython-input-72-b64ce99dceb4>:8: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory()
<ipython-input-72-b64ce99dceb4>:9: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :meth:`~RunnableWithMessageHistory: https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html` instead.
  conversation = ConversationChain(




> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Query: What is the stock  that i can buy for $1000 over 5 years at 5% annual rate?
Available functions: calculate_compound_interest, get_stock_price, convert_currency
AI:

> Finished chain.


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Query: What is the stock  that i can buy for $1000 over 5 years at 5% annual rate?
Available functions: calculate_compound_interest, get_stock_price,

##  Advanced Concepts in Function Calling

### Multi-step Reasoning

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

reasoning_prompt = PromptTemplate(
    input_variables=["query", "available_functions"],
    template="Query: {query}\nAvailable functions: {available_functions}\nBreak down the steps needed to answer this query using the available functions."
)

reasoning_chain = LLMChain(llm=llm, prompt=reasoning_prompt)

def multi_step_function_calling(query, available_functions):
    steps = reasoning_chain.run(query=query, available_functions=available_functions)
    # Implement logic to execute each step and aggregate results
    return steps

query = "What's the price difference between Apple and Microsoft stocks over the last month?"
available_functions = ["get_stock_price", "calculate_percentage_change", "get_date_range"]
execution_plan = multi_step_function_calling(query, available_functions)
execution_plan

'\n\nStep 1: Use the get_date_range function to get the date range of the last month.\n\nStep 2: Use the get_stock_price function to get the stock prices of Apple and Microsoft on the starting and ending dates of the last month.\n\nStep 3: Calculate the percentage change of Apple and Microsoft stocks using the calculate_percentage_change function.\n\nStep 4: Subtract the percentage change of Apple from the percentage change of Microsoft to find the price difference between the two stocks.\n\nStep 5: Print or display the result to the user.'

# **End Of Chapter 2**